In [5]:
import mysql.connector
import psycopg2
import pandas as pd
import time
import numpy as np
from config import host, user, password
from config2 import user1, password1

In [6]:
start_time = time.time()

In [7]:
### Establishing MySQL and PostgreSQL Connections

#MySQL Connection
try:
    mysql_db = mysql.connector.connect(host=host,user=user,password=password,db="eBooks2go")
    mycursor = mysql_db.cursor()
    print("MySQL Connection Established")
except mysql.connector.Error as e:
    print("Unable to Connect: ",format(e))

#PostgreSQL Connection
try:
    pg_db = psycopg2.connect(host=host,user=user1,password=password1,database="staging")
    pg_db.autocommit = True
    pgcursor = pg_db.cursor()
    print("PostgreSQL Connection Established")
except psycopg2.OperationalError as e:
    print("Unable to Connect: ",format(e))

    

MySQL Connection Established
PostgreSQL Connection Established


In [ ]:
count_success = 0
Total_Records_from_source = 0
count_failed_emails = 0

try: 
    #Selects data from table
    data = pd.read_excel(r'/Users/mahi/Gantech/June_Overdrive.xlsx')
    data.columns = data.columns.str.replace(' ', '_')
    
    Total_Records_from_source += data.shape[0]

    #Extracting column names 
    col_names_lst = data.columns.tolist()
    col_names = ', '.join(col_names_lst)+', Created_Date'+', Updated_Date'

    #Inserting data from MySQL into PostgreSQL
    placeholders = ', '.join(['%s'] * (len(col_names_lst)+2) ) 
    # print(placeholders)

    #Inserting data into PostgreSQL
    insrt_cmd = "INSERT INTO stg_overdrive_sales("+col_names+") " \
                "VALUES ( %s )" % (placeholders)
    # print(insrt_cmd)

    for row in data.itertuples(index=False).unique():
        pgcursor.execute(insrt_cmd,tuple(row)+tuple([time.strftime("%Y/%m/%d %H:%M")])+tuple([time.strftime("%Y/%m/%d %H:%M")]))
        count_success += 1
        
except Exception as e:
    count_failed_emails += 1  
    
insrt_log = "INSERT INTO LOG_DIM (Phase, Source_Table_Name, Target_Table_Name, Total_Records_Source_Table,Total_Records_Target_Table, Status, Remarks, Execution_time) Values ('LtoS','Overdrive_Excel','stg_overdrive_sales'," +str(Total_Records_from_source)+", "+str(count_success)+",'Completed','" +str(count_failed_emails)+" Failed','"+str(round(time.time() - start_time,2))+" seconds')"
pgcursor.execute(insrt_log)

print("Done!")

pg_db.close()
    